In [21]:
import cPickle as pickle

import scipy
import numpy as np
import pandas as pd
import re

# Load English stopwords from NLTK
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize, RegexpTokenizer, WhitespaceTokenizer
import nltk.data

from gensim.models import word2vec
import logging

import os

print ("Required Libraries loaded.")

Required Libraries loaded.


In [22]:
# Import articles for string cleaning

def clean_text(text, stem_words = False, remove_stopwords = False):
    ###clean
    texts = re.sub("[^a-zA-Z]",    #pattern to match
              " ",              #replace other with this
              text)                 #text to apply to
         
    #print "Text recived: ", texts 
    clean_corpus = texts.lower().split()
    #print "corpus: ", clean_corpus
    if stem_words:
        # Porter stemmer
        porter = nltk.PorterStemmer()
        # Snowball stemmer
        snowball = nltk.SnowballStemmer('english')
        # Lancaster stemmer
        lancaster = nltk.LancasterStemmer()
        # General stemming Lambda function to stem tokens
        clean_corpus = lambda tokens: [porter.stem(w) for w in corpus]
    if remove_stopwords:   # Optionally remove stop words
        stops = set(stopwords.words("english"))
        clean_corpus = [w for w in clean_corpus if not w in stops]
        #print "Clean_corpus: ", clean_corpus
    
    return (clean_corpus)

In [23]:
# Use the punkt tokenizer for sentence splitting

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def article_to_sentences(article, tokenizer, stem_words=False, remove_stopwords=False):
    """
    article - article to use as input to create the wordlist
    tokenizer - the tokenizer to use to split into sentences
    stem_words - boolean, whether to use the stemmer function or not
    remove_stopwords - boolean, whether to remove the stopwords function or not
    
    article_to_sentences: Function to convert a document to a sequence of sentences,
    optionally removing stop words.  
    returns: a sequence of sentences where each sentence is itself a sequence of words
    """
    raw_sentences = tokenizer.tokenize(article.decode('utf-8').strip())   # Punkt tokenize into sentences
    sentences = []   # create list of sentences 
    for raw_sentence in raw_sentences:
        #print "raw sentence: ", raw_sentence
        #print len(raw_sentence)
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(clean_text(raw_sentence, stem_words, remove_stopwords))

    # returns a list of lists: list of sentences composed of lists of words
        #print "Sentences: ", sentences
    return sentences

In [24]:
from __future__ import unicode_literals
def create_sentences(corpus, name):
    print ("Begin sentences creation %s..." % name)
    sentences = []  # Initialize an empty list of sentences

    print ("Parsing sentences from training set")
    for article in corpus:
        sentences += article_to_sentences(article, tokenizer)
    print ("Finished data loading and creating sentences")
    return sentences

In [25]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
def train_save_model(sentences, filename):

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

    # Set values for various parameters
    #num_features = 210    # Word vector dimensionality                      
    min_word_count = 10   # Minimum word count                        
    num_workers = 8       # Number of threads to run in parallel
    context = 15          # Context window size                                                                                    
    downsampling = 1e-3   # Downsample setting for frequent words

    # Initialize and train the model (this will take some time)

    print ("Training model...")
    if (filename == "ACEInhibitors"):
        num_features = 210
    elif (filename == "ADHD"):
        num_features = 80
    elif (filename == "Antihistamines"):
        num_features = 29
    elif (filename == "AtypicalAntipsychotics"):
        num_features = 381
    elif (filename == "BetaBlockers"):
        num_features = 194
    elif (filename == "CalciumChannelBlockers"):
        num_features = 329
    elif (filename == "Estrogens"):
        num_features = 233
    elif (filename == "NSAIDS"):
        num_features = 242
    elif (filename == "Opiods"):
        num_features = 55
    elif (filename == "OralHypoglycemics"):
        num_features = 234
    elif (filename == "ProtonPumpInhibitors"):
        num_features = 206
    elif (filename == "SkeletalMuscleRelaxants"):
        num_features = 11
    elif (filename == "Statins"):
        num_features = 467
    elif (filename == "Triptans"):
        num_features = 121
    elif (filename == "UrinaryIncontinence"):
        num_features = 215
    model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, 
                              window = context, sample = downsampling)

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)

    # It can be helpful to create a meaningful model name and 
    # save the model for later use. You can load it later using Word2Vec.load()
    
    model_name = filename + "_chi2_features_10minwords_15context"
    folder = "C:/EPC_Data/complete_data/word2vec"
    print ("Saving the model: %s...." % model_name)
    model_name = os.path.join(folder, model_name)
    model.save(model_name)

    print ("Finished Model training and saving for %s...") % filename

In [26]:
#iterate through a directory, load a file, sentence = filename.TiAbsMesh
from __future__ import unicode_literals
path = "C:/EPC_Data/complete_data/raw"
for file in os.listdir(path):
    current_file = os.path.join(path, file)
    filename, _ = file.split(".")
        
    file_obj = pd.read_csv(current_file, sep=",", index_col='PMID')
    corpus = file_obj.TiAbsMesh
   
    train_save_model(create_sentences(corpus, filename), filename)

Begin sentences creation ACEInhibitors...
Parsing sentences from training set


2017-04-03 15:55:54,043 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2017-04-03 15:55:54,059 : INFO : collecting all words and their counts
2017-04-03 15:55:54,059 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-03 15:55:54,101 : INFO : PROGRESS: at sentence #10000, processed 218649 words, keeping 7797 word types
2017-04-03 15:55:54,142 : INFO : PROGRESS: at sentence #20000, processed 436426 words, keeping 10410 word types
2017-04-03 15:55:54,180 : INFO : collected 11827 word types from a corpus of 617799 raw words and 28186 sentences
2017-04-03 15:55:54,180 : INFO : Loading a fresh vocabulary
2017-04-03 15:55:54,237 : INFO : min_count=10 retains 3441 unique words (29% of original 11827, drops 8386)


Finished data loading and creating sentences
Training model...


2017-04-03 15:55:54,253 : INFO : min_count=10 leaves 595078 word corpus (96% of original 617799, drops 22721)
2017-04-03 15:55:54,253 : INFO : deleting the raw counts dictionary of 11827 items
2017-04-03 15:55:54,253 : INFO : sample=0.001 downsamples 58 most-common words
2017-04-03 15:55:54,253 : INFO : downsampling leaves estimated 452196 word corpus (76.0% of prior 595078)
2017-04-03 15:55:54,269 : INFO : estimated required memory for 3441 words and 210 dimensions: 7501380 bytes
2017-04-03 15:55:54,269 : INFO : resetting layer weights
2017-04-03 15:55:54,328 : INFO : training model with 8 workers on 3441 vocabulary and 210 features, using sg=0 hs=0 sample=0.001 negative=5 window=15
2017-04-03 15:55:54,329 : INFO : expecting 28186 sentences, matching count from corpus used for vocabulary survey
2017-04-03 15:55:55,332 : INFO : PROGRESS: at 71.10% examples, 1607488 words/s, in_qsize 0, out_qsize 1
2017-04-03 15:55:55,765 : INFO : worker thread finished; awaiting finish of 7 more thread

Saving the model: ACEInhibitors_chi2_features_10minwords_15context....
Finished Model training and saving for ACEInhibitors...
Begin sentences creation ADHD...
Parsing sentences from training set


...

Saving the model: Triptans_chi2_features_10minwords_15context....
Finished Model training and saving for Triptans...
Begin sentences creation UrinaryIncontinence...
Parsing sentences from training set


2017-04-03 15:56:17,289 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2017-04-03 15:56:17,292 : INFO : collecting all words and their counts
2017-04-03 15:56:17,292 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-03 15:56:17,305 : INFO : collected 5386 word types from a corpus of 72183 raw words and 3664 sentences
2017-04-03 15:56:17,308 : INFO : Loading a fresh vocabulary
2017-04-03 15:56:17,312 : INFO : min_count=10 retains 1003 unique words (18% of original 5386, drops 4383)
2017-04-03 15:56:17,313 : INFO : min_count=10 leaves 60999 word corpus (84% of original 72183, drops 11184)
2017-04-03 15:56:17,318 : INFO : deleting the raw counts dictionary of 5386 items
2017-04-03 15:56:17,319 : INFO : sample=0.001 downsamples 55 most-common words
2017-04-03 15:56:17,319 : INFO : downsampling leaves estimated 43743 word corpus (71.7% of prior 60999)
2017-04-03 15:56:17,321 : INFO : estimated required memory for 1003 word

Finished data loading and creating sentences
Training model...


2017-04-03 15:56:17,500 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-04-03 15:56:17,500 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-04-03 15:56:17,500 : INFO : training on 360915 raw words (218491 effective words) took 0.2s, 1425940 effective words/s
2017-04-03 15:56:17,500 : WARNING : under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-04-03 15:56:17,500 : INFO : precomputing L2-norms of word weight vectors
2017-04-03 15:56:17,500 : INFO : saving Word2Vec object under C:/EPC_Data/complete_data/word2vec\UrinaryIncontinence_chi2_features_10minwords_15context, separately None
2017-04-03 15:56:17,515 : INFO : not storing attribute syn0norm
2017-04-03 15:56:17,516 : INFO : not storing attribute cum_table
2017-04-03 15:56:17,542 : INFO : saved C:/EPC_Data/complete_data/word2vec\UrinaryIncontinence_chi2_features_10minwords_15context


Saving the model: UrinaryIncontinence_chi2_features_10minwords_15context....
Finished Model training and saving for UrinaryIncontinence...
